In [66]:
import fun_tabla_pdf
import pandas as pd
import requests

In [67]:
pdf_nombre = 'Randstad_Julio_2025.pdf'

# colocar la primera y ultima pagina donde aparecen tablas
inicio = 11
fin = 79

l_pag = [i for i in range(inicio , fin + 1)]
todos_df = []
todos_df_unpivot = []

for i_pag in l_pag:
    pag = i_pag

    #salta la iteracion si la tabla de datos esta vacia, es porque se trata de un pagina con solo el titulo de area profesional
    if fun_tabla_pdf.sin_tabla(pdf_nombre,pag):
        print('Pagina sin tabla: ', pag + 1)
        continue

    # Extraccion de datos de numeros y nombre de columnas de zona (zona derecha de la tabla en pdf)
    datos_tabla = fun_tabla_pdf.tabla_datos_num(pdf_nombre,pag)
    datos_tabla_stg1 = fun_tabla_pdf.filtrar_items(datos_tabla)
    datos_tabla_stg1 = fun_tabla_pdf.f_bubble_sort(datos_tabla_stg1)
    datos_tabla_stg2 = fun_tabla_pdf.datos_a_fila(datos_tabla_stg1)
    l_datos_tablas = fun_tabla_pdf.datos_a_tabla(datos_tabla_stg2)

    #Extraccion de datos de las cabeceras/categorias/puesto del lado izquierdo del pdf
    encabezados = fun_tabla_pdf.encabezados_datos(pdf_nombre , pag)
    encabezados_stg1 = fun_tabla_pdf.f_bubble_sort(encabezados)
    encabezados_stg2 = fun_tabla_pdf.f_misma_fila(encabezados_stg1)
    encabezados_stg3, encabezados_stg3_11 = fun_tabla_pdf.separacion_encabezado(l_datos_tablas, encabezados_stg2)

    #Extraccion del dato de area profesional, zona de cabecera de pagina del pdf
    area_profesional = fun_tabla_pdf.area_profesional(pdf_nombre, pag)

    #Uniendo todo en una sola tabla y convertir en df
    lista_l_df = fun_tabla_pdf.unir_todo(l_datos_tablas, encabezados_stg3 ,encabezados_stg3_11 ,area_profesional)
    df_tables, df_tables_unpivot = fun_tabla_pdf.generar_df(lista_l_df)

    for table in df_tables:
        todos_df.append(table)
    
    for table_unpv in df_tables_unpivot:
        todos_df_unpivot.append(table_unpv)
    

Pagina sin tabla:  12
Pagina sin tabla:  16
Pagina sin tabla:  30
Pagina sin tabla:  36
Pagina sin tabla:  39
Pagina sin tabla:  42
Pagina sin tabla:  44
Pagina sin tabla:  46
Pagina sin tabla:  48
Pagina sin tabla:  51
Pagina sin tabla:  54
Pagina sin tabla:  57
Pagina sin tabla:  60
Pagina sin tabla:  70
Pagina sin tabla:  76


In [68]:
df_concat = pd.concat(todos_df)
df_concat_unpivot = pd.concat(todos_df_unpivot)

#### Consultamos la api argentinadatos para calcular el dolar promedio para Julio de 2025, que es el mes del pdf

In [69]:
text = requests.get(
    "https://api.argentinadatos.com/v1/cotizaciones/dolares"
)
l_aux = []

for i in text.json():
    if i['casa'] == 'oficial' and '2025-07' in i['fecha'] :
        v_dolar_venta = int(i['venta'])
        l_aux.append( v_dolar_venta )

v_promedio = round(sum(l_aux) / len(l_aux))


In [70]:
df_concat['Dolar_Promedio_Mes'] = v_promedio
df_concat_unpivot['Dolar_Promedio_Mes'] = v_promedio

#### Exportando a csv

In [71]:
df_concat.to_csv('DF_TABLAS.csv', index = False, encoding = 'utf-8-sig', sep = ";")
df_concat_unpivot.to_csv('DF_TABLAS_UNPIVOT.csv', index = False, encoding = 'utf-8-sig', sep = ";")